In [ ]:
!pip install rectools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.0/89.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 16.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.0/89.0 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 23.0 MB/s eta 0:00:00
  Created wheel for implicit: filename=implicit-0.4.4-cp39-cp39-linux_x86_64.whl size=5054331 sha256=155170e6fb4b5dd3704dde1656b5f1ad4e6edc8c958398aefd96134f79c895fa
  Stored in directory: /root/.cache/pip/wheels/05/d3/5d/ab371b92ddb01183c3997751ad3bed6e6a

In [ ]:
!pip install hnswlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for hnswlib: filename=hnswlib-0.7.0-cp39-cp39-linux_x86_64.whl size=2118378 sha256=25359ee4b9964395eb8fe8cdc00bc935aa5a67d1731d519b2dd0dab0e655e0e0
  Stored in directory: /root/.cache/pip/wheels/ba/26/61/fface6c407f56418b3140cd7645917f20ba6b27d4e32b2bd20
Successfully built hnswlib


In [ ]:
!pip install lightfm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from random import randint
import sys

import requests
from lightfm import LightFM
import pandas as pd
import numpy as np
from rectools.metrics import Precision, Recall, MAP, calc_metrics
from rectools import Columns
from rectools.dataset import Dataset
from rectools.models import LightFMWrapperModel

# LOAD DATA

In [ ]:
!wget https://storage.yandexcloud.net/itmo-recsys-public-data/kion_train.zip

--2023-04-15 21:14:05--  https://storage.yandexcloud.net/itmo-recsys-public-data/kion_train.zip
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 78795385 (75M) [application/zip]
Saving to: ‘kion_train.zip’

kion_train.zip      100%[===================>]  75.14M  23.4MB/s    in 3.9s    

2023-04-15 21:14:10 (19.5 MB/s) - ‘kion_train.zip’ saved [78795385/78795385]



In [ ]:
!unzip kion_train.zip

Archive:  kion_train.zip
   creating: kion_train/
  inflating: kion_train/interactions.csv  
  inflating: __MACOSX/kion_train/._interactions.csv  
  inflating: kion_train/users.csv    
  inflating: __MACOSX/kion_train/._users.csv  
  inflating: kion_train/items.csv    
  inflating: __MACOSX/kion_train/._items.csv  


In [ ]:
interactions = pd.read_csv('kion_train/interactions.csv')
users = pd.read_csv('kion_train/users.csv')
items = pd.read_csv('kion_train/items.csv')

# Preprocess

In [ ]:
Columns.Datetime = 'last_watch_dt'

In [ ]:
interactions.drop(interactions[interactions[Columns.Datetime].str.len() != 10].index, inplace=True)

In [ ]:
interactions[Columns.Datetime] = pd.to_datetime(interactions[Columns.Datetime], format='%Y-%m-%d')

In [ ]:
max_date = interactions[Columns.Datetime].max()

In [ ]:
interactions[Columns.Weight] = np.where(interactions['watched_pct'] > 10, 3, 1)

In [ ]:
train = interactions[interactions[Columns.Datetime] < max_date - pd.Timedelta(days=7)].copy()
test = interactions[interactions[Columns.Datetime] >= max_date - pd.Timedelta(days=7)].copy()

print(f"train: {train.shape}")
print(f"test: {test.shape}")

train: (4985269, 6)
test: (490982, 6)


In [ ]:
train.drop(train.query("total_dur < 300").index, inplace=True)

In [ ]:
# отфильтруем холодных пользователей из теста
cold_users = set(test[Columns.User]) - set(train[Columns.User])

In [ ]:
test.drop(test[test[Columns.User].isin(cold_users)].index, inplace=True)

# Models

In [ ]:
TEST_USERS = test[Columns.User].unique()

# Добавляем аватаров в обучающую выборку

1. Девушка, которая смотрит только мелодрамы
3. Девочка, смотрит только аниме, хочет уехать в Японию.
3. Фанат Райана Гослинга

In [ ]:
dramma_queen = items[(items.genres.isna() == False) & (items.genres.str.contains('мелодрамы'))][:10]
anime_tyan = items[(items.age_rating <= 12) & (items.keywords.str.contains('аниме', case=False))][:15]
gosling_fan = items[(items.actors.isna() == False) & (items.actors.str.contains('Райан Гослинг'))][:10]

new_items = [dramma_queen, anime_tyan, gosling_fan]

In [ ]:
max_user_id = train.user_id.max()
avatar_ids = list()

for user_items in new_items:
    max_user_id += 1
    for item_id in user_items.item_id:
        last_watch_dt = f'2022-{str(randint(1,12)).zfill(2)}-{str(randint(1,28)).zfill(2)}'
        if max_user_id not in avatar_ids:
            avatar_ids.append(max_user_id)
        train = train.append({
            'user_id': max_user_id,
            'item_id': item_id,
            'last_watch_dt': last_watch_dt,
            'watched_pct': randint(70, 100),
            'weight': 3
        }, ignore_index=True)

<ipython-input-19-3fb76f32fe70>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train = train.append({
<ipython-input-19-3fb76f32fe70>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train = train.append({
<ipython-input-19-3fb76f32fe70>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train = train.append({
<ipython-input-19-3fb76f32fe70>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train = train.append({
<ipython-input-19-3fb76f32fe70>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train = train.append({
<ipython-input-19-3fb76f32fe70

In [ ]:
avatar_map = pd.DataFrame({"user_id": avatar_ids, "name": ['dramma_queen', 'anime_tyan', 'gosling_fan']})

In [ ]:
avatar_ids

[1097558, 1097559, 1097560]

# Обучение LightFm

In [ ]:
dataset = Dataset.construct(interactions_df=train)

/usr/local/lib/python3.9/dist-packages/rectools/dataset/interactions.py:62: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  df[Columns.Datetime] = df[Columns.Datetime].astype("datetime64[ns]")


In [ ]:
model = LightFMWrapperModel(
        LightFM(
            no_components=12,
            loss='warp',
            random_state=1234,
        ),
        epochs=10,
        num_threads=4,
    )

model.fit(dataset)

In [ ]:
recoms = model.recommend(
    users=avatar_ids,
    dataset=dataset,
    k=10,
    filter_viewed=True,
)
recoms = pd.merge(recoms, items, on='item_id')
recoms = pd.merge(recoms, avatar_map, on='user_id')[['name', 'countries', 'title', 'genres', 'age_rating', 'actors', 'keywords']]


# Approximate Nearest Neighbors

In [ ]:
import hnswlib
import nmslib

In [ ]:
from pydantic import BaseSettings
class ApproximateNearestNeighbors:
    def __init__(
        self,
        model,
        dataset: Dataset,
        M: int = 256,
        efC: int = 128,
        efS: int = 256,
        num_threads = 8,
    ):
        self.model = model
        self.dataset = dataset
        self.num_threads = num_threads
        self.index = nmslib.init(
            method='hnsw',
            space='negdotprod',
            data_type=nmslib.DataType.DENSE_VECTOR
        )
        self.index_time_params = {
            'M': M,
            'indexThreadQty': self.num_threads,
            'efConstruction': efC
        }
        self.neighbours = None
        self.query_time_params = {'efSearch': efS}

    @staticmethod
    def augment_inner_product(factors):
        normed_factors = np.linalg.norm(factors, axis=1)
        max_norm = normed_factors.max()

        extra_dim = np.sqrt(max_norm ** 2 - normed_factors ** 2).reshape(-1, 1)
        augmented_factors = np.append(factors, extra_dim, axis=1)
        return augmented_factors

    def fit(self, k_reco: int = 10):
        user_embeddings, item_embeddings = self.model.get_vectors(self.dataset)
        augmented_item_embeddings = self.augment_inner_product(
            item_embeddings
        )
        extra_zero = np.zeros((user_embeddings.shape[0], 1))
        augmented_user_embeddings = np.append(
            user_embeddings,
            extra_zero,
            axis=1
        )

        self.index.addDataPointBatch(augmented_item_embeddings)
        self.index.createIndex(self.index_time_params)
        self.index.setQueryTimeParams(self.query_time_params)

        self.neighbours = self.index.knnQueryBatch(
            augmented_user_embeddings,
            k=k_reco,
            num_threads=self.num_threads
        )

    def predict(self, user_id: int):
        int_id = self.dataset.user_id_map.convert_to_internal([user_id])[0]
        user_recs = self.neighbours[int_id][0]  # type: ignore
        return self.dataset.item_id_map.convert_to_external(user_recs)

In [ ]:
ann = ApproximateNearestNeighbors(model=model, dataset=dataset)
ann.fit(k_reco=10)

In [ ]:
user_embeddings, item_embeddings = model.get_vectors(dataset)

In [ ]:
user_embeddings.shape, item_embeddings.shape

((756565, 14), (14020, 14))

In [ ]:
def augment_inner_product(factors):
    normed_factors = np.linalg.norm(factors, axis=1)
    max_norm = normed_factors.max()
    
    extra_dim = np.sqrt(max_norm ** 2 - normed_factors ** 2).reshape(-1, 1)
    augmented_factors = np.append(factors, extra_dim, axis=1)
    return max_norm, augmented_factors

In [ ]:
ann.predict

## Драма квинн
Модель не так уж плохо справилась, в плане предсказания фильмов, всего 1 мелодрама и 3 драмы

In [ ]:
lightfm_recs_dramma_queen = recoms[:10]
ann_recs_dramma_queen = items[items.item_id.isin(ann.predict(user_id=avatar_ids[0]))]

In [ ]:
assert set(lightfm_recs_dramma_queen.title) == set(ann_recs_dramma_queen.title)

In [ ]:
ann_recs_dramma_queen

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
767,15297,series,Клиника счастья,Klinika schast'ya,2021.0,"драмы, мелодрамы",Россия,NaN,18.0,NaN,Александр Кириенко,"Дарья Мороз, Анатолий Белый, Данил Акутин, Мар...","Успешный сексолог Алена уверена, что нашла фор...","Клиника счастья, Клиника, Счастье, Клиника сча..."
1882,10440,series,Хрустальный,Khrustal'nyy,2021.0,"триллеры, детективы",Россия,NaN,18.0,NaN,Душан Глигоров,"Антон Васильев, Николай Шрайбер, Екатерина Оль...",Сергей Смирнов — один из лучших «охотников на ...,"хруст, хрусталь, хруста, хрус, полицейский, пе..."
3875,6809,film,Дуров,NaN,2021.0,документальное,Россия,NaN,16.0,NaN,Родион Чепель,NaN,"Уникальная история о лидере нового поколения, ...","Компьютер, Монитор, Гений, Интервью, Предприни..."
6346,9728,film,Гнев человеческий,Wrath of Man,2021.0,"боевики, триллеры","Великобритания, США",NaN,18.0,NaN,Гай Ричи,"Джейсон Стэйтем, Холт МакКэллани, Джеффри Доно...",Грузовики лос-анджелесской инкассаторской комп...,"ограбление, криминальный авторитет, месть, пер..."
6501,13865,film,Девятаев,V2. Escape from Hell,2021.0,"драмы, военные, приключения",Россия,NaN,12.0,NaN,Тимур Бекмамбетов,"Павел Прилучный, Павел Чинарёв, Тимофей Трибун...",Военно-исторический блокбастер от режиссёров Т...,"Девятаев, Девятаева, Девят, Девя, Девята, Девя..."
6689,4151,series,Секреты семейной жизни,NaN,2021.0,комедии,Россия,NaN,18.0,NaN,Шота Гамисония,"Петр Скворцов, Алена Михайлова, Федор Лавров, ...",У Никиты и Полины всё начиналось прекрасно: об...,"брызги крови, кровь, жестокое обращение с живо..."
7486,8636,film,Белый снег,Belyy sneg,2021.0,"драмы, спорт",Россия,NaN,6.0,NaN,Николай Хомерики,"Ольга Лерман, Федор Добронравов, Надежда Марки...",Отечественная спортивная драма про лыжницу и о...,"биография, занятие спортом, байопик, чемпионат..."
9253,1844,film,Аферистка,I Care A Lot,2020.0,"триллеры, комедии",Великобритания,NaN,18.0,NaN,Джей Блейксон,"Розамунд Пайк, Питер Динклэйдж, Эйса Гонсалес,...",Марла руководствуется принципом «побеждает сил...,"опекун, аферист, чёрная комедия, мошенник, 202..."
12050,3734,film,Прабабушка легкого поведения,Prababushka lyogkogo povedeniya,2021.0,комедии,Россия,NaN,16.0,NaN,Марюс Вайсберг,"Александр Ревва, Глюкоза, Дмитрий Нагиев, Миха...","1980 год, вся страна следит за событиями моско...",", 2021, россия, прабабушка, легкого, поведения"
15234,142,film,Маша,Masha,2020.0,"драмы, триллеры",Россия,NaN,16.0,NaN,Анастасия Пальчикова,"Максим Суханов, Аня Чиповская, Полина Гухман, ...",13-летняя Маша растёт между боксёрским рингом ...,"Фильм Маша, Маша фильм 2021, Смотреть фильм Ма..."


In [ ]:
recoms

## Аниме-тян
Отличные рекомендации, 4 аниме

In [ ]:
lightfm_recs_anime_tyan = recoms[10:20]
ann_recs_anime_tyan = items[items.item_id.isin(ann.predict(user_id=avatar_ids[1]))]

In [ ]:
ann_recs_anime_tyan

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
1274,5411,film,Монстры на каникулах 3: Море зовёт,Hotel Transylvania 3: Summer Vacation,2018.0,"мультфильм, фэнтези, приключения, комедии",США,NaN,6.0,NaN,Генндий Тартаковский,"Энди Сэмберг, Селена Гомес, Кевин Джеймс, Стив...",На этот раз Дракула вместе со своими друзьями ...,"третья часть, круизное судно, персонаж Дракула..."
1554,16270,film,Тайна Коко,Coco,2017.0,"мультфильм, фэнтези, приключения",США,NaN,12.0,NaN,"Ли Анкрич, Эдриан Молина","Энтони Гонсалес, Гаэль Гарсиа Берналь, Бенджам...",Мексиканский юноша Мигель живёт в семье сапожн...,"Мексика, гитара, музыкант, скелет, музыка, заг..."
1849,7571,film,100% волк,100% Wolf,2020.0,"мультфильм, приключения, семейное, фэнтези, ко...","Австралия, Бельгия",NaN,6.0,NaN,Алекс Стадерманн,"Илай Суинделлс, Самара Уивинг, Джай Кортни, Ру...",Наследник семьи оборотней Фредди Люпин отчаянн...,"пудель, подростковая тревога, оборотень, приня..."
3951,3182,film,Ральф против Интернета,Ralph Breaks the Internet,2018.0,"мультфильм, приключения, фантастика, семейное,...",США,NaN,6.0,NaN,"Рич Мур, Фил Джонстон","Джон Си Райли, Сара Силверман, Галь Гадот, Тар...",На этот раз Ральф и Ванилопа фон Кекс выйдут з...,"видеоигра, мультфильм, продолжение, интернет, ..."
5206,13159,film,Рататуй,Ratatouille,2007.0,"мультфильм, приключения, драмы, фэнтези, комедии",США,NaN,0.0,NaN,"Брэд Бёрд, Ян Пинкава","Пэттон Освальт, Иэн Холм, Лу Романо, Брайан Де...",Крыс Реми обладает уникальным вкусом. Он готов...,"Париж, Франция, родственные отношения, дорогой..."
8275,2954,film,Миньоны,Minions,2015.0,"фантастика, мультфильм, приключения, комедии",США,NaN,6.0,NaN,"Кайл Балда, Пьер Коффан","Сандра Буллок, Джон Хэмм, Майкл Китон, Эллисон...",Миньоны живут на планете гораздо дольше нас. У...,"помощник, сцена после титров, сцена во время т..."
9698,16166,film,Зверополис,Zootopia,2016.0,"приключения, мультфильм, детективы, комедии",США,NaN,6.0,NaN,"Байрон Ховард, Рич Мур, Джаред Буш","Джиннифер Гудвин, Джейсон Бейтман, Идрис Эльба...",Добро пожаловать в Зверополис — современный го...,"аллегория, лев, бегемот, лиса, слон, овца, бел..."
11254,11749,film,Суперсемейка 2,Incredibles 2,2018.0,"фантастика, мультфильм, приключения",США,NaN,6.0,NaN,Брэд Бёрд,"Крэйг Т. Нельсон, Холли Хантер, Сара Вауэлл, Х...",После событий первой части суперсемейка пользу...,"семейная пара, продолжение, супергерой, дети, ..."
12471,14942,film,История игрушек: Большой побег,Toy Story 3,2010.0,"мультфильм, фэнтези, комедии",США,NaN,6.0,NaN,Ли Анкрич,"Том Хэнкс, Тим Аллен, Джоан Кьюсак, Нед Битти,...","Энди почти 18 лет, ему остаётся 3 дня до отпра...","заложник, колледж, игрушка, побег, детский сад..."
12620,13243,film,Головоломка,Inside Out,2015.0,"фантастика, мультфильм, комедии",США,NaN,6.0,NaN,"Пит Доктер, Роналдо Дель Кармен","Эми Полер, Филлис Смит, Ричард Кайнд, Билл Хей...",11-летняя девочка Райли переезжает с семьёй из...,"мечта, мультфильм, воображаемый друг, начальна..."


## Фанат Райана Гослинга
В рекомендациях ни 1 фильма с Райаном гослингом. Но что-то схожее с жанром было рекомендовано. Практически нет зарубежных фильмов.

In [ ]:
lightfm_recs_gosling_fan = recoms[20:30]
ann_recs_gosling_fan = items[items.item_id.isin(ann.predict(user_id=avatar_ids[2]))]

In [ ]:
ann_recs_gosling_fan

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
767,15297,series,Клиника счастья,Klinika schast'ya,2021.0,"драмы, мелодрамы",Россия,NaN,18.0,NaN,Александр Кириенко,"Дарья Мороз, Анатолий Белый, Данил Акутин, Мар...","Успешный сексолог Алена уверена, что нашла фор...","Клиника счастья, Клиника, Счастье, Клиника сча..."
927,11778,film,Простые сложности,It's Complicated,2009.0,"драмы, мелодрамы, комедии","США, Япония",NaN,16.0,NaN,Нэнси Майэрс,"Мэрил Стрип, Алек Болдуин, Стив Мартин, Джон К...","Режиссёрка Нэнси Майерс, известная благодаря ф...","Нью-Йорк, отель, жених, выпускной, бывший муж,..."
1882,10440,series,Хрустальный,Khrustal'nyy,2021.0,"триллеры, детективы",Россия,NaN,18.0,NaN,Душан Глигоров,"Антон Васильев, Николай Шрайбер, Екатерина Оль...",Сергей Смирнов — один из лучших «охотников на ...,"хруст, хрусталь, хруста, хрус, полицейский, пе..."
3875,6809,film,Дуров,NaN,2021.0,документальное,Россия,NaN,16.0,NaN,Родион Чепель,NaN,"Уникальная история о лидере нового поколения, ...","Компьютер, Монитор, Гений, Интервью, Предприни..."
6346,9728,film,Гнев человеческий,Wrath of Man,2021.0,"боевики, триллеры","Великобритания, США",NaN,18.0,NaN,Гай Ричи,"Джейсон Стэйтем, Холт МакКэллани, Джеффри Доно...",Грузовики лос-анджелесской инкассаторской комп...,"ограбление, криминальный авторитет, месть, пер..."
6501,13865,film,Девятаев,V2. Escape from Hell,2021.0,"драмы, военные, приключения",Россия,NaN,12.0,NaN,Тимур Бекмамбетов,"Павел Прилучный, Павел Чинарёв, Тимофей Трибун...",Военно-исторический блокбастер от режиссёров Т...,"Девятаев, Девятаева, Девят, Девя, Девята, Девя..."
6689,4151,series,Секреты семейной жизни,NaN,2021.0,комедии,Россия,NaN,18.0,NaN,Шота Гамисония,"Петр Скворцов, Алена Михайлова, Федор Лавров, ...",У Никиты и Полины всё начиналось прекрасно: об...,"брызги крови, кровь, жестокое обращение с живо..."
9253,1844,film,Аферистка,I Care A Lot,2020.0,"триллеры, комедии",Великобритания,NaN,18.0,NaN,Джей Блейксон,"Розамунд Пайк, Питер Динклэйдж, Эйса Гонсалес,...",Марла руководствуется принципом «побеждает сил...,"опекун, аферист, чёрная комедия, мошенник, 202..."
12050,3734,film,Прабабушка легкого поведения,Prababushka lyogkogo povedeniya,2021.0,комедии,Россия,NaN,16.0,NaN,Марюс Вайсберг,"Александр Ревва, Глюкоза, Дмитрий Нагиев, Миха...","1980 год, вся страна следит за событиями моско...",", 2021, россия, прабабушка, легкого, поведения"
14889,4740,film,Сахаров. Две жизни,NaN,2021.0,документальное,Россия,NaN,16.0,NaN,Иван Проскуряков,"Алексей Усольцев, Чулпан Хаматова, Агата Супер","Его имя носят проспект, университет и даже аст...","Сахаров, Сахарок, Сахар, Сахар сахаров, Сахаро..."


ВЫВОД: Получили быстрее работающие предикты, без потери качества предсказаний, модель можно использовать для онлайн подхода